In [36]:
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")

Aggregate

In [37]:
@make_struct_func Cubic

force_func (generic function with 1 method)

## Add Data

In [38]:
parameters = Model(
    Cubic(
        μ₁      = 0.1,
        rₘᵢₙ    = 2,
        rₘₐₓ    = 3
    ),
    Contractile(
        fₚ      = 0.0000001
    ),
    Time(
        t_f     = 100000, 
        dt      = 0.1
    ),
    Neighbor(
        n_knn   = 50,
        nn      = 12
    ),
    Geometry(
        R_agg   = 9,
        num_agg = 2
    ),
    Simulation(
        n_text  = 200
    )
)

Model(Cubic(0.1, 2.0, 3.0), Contractile(1.0e-7), Time(0.0, 100000.0, 0.1), Neighbor(50, 12), Geometry(9.0, 2), Simulation(200))

In [39]:
X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

In [40]:
## Var before func
force(r) =  force_func(parameters.Force::Cubic, r)

force (generic function with 1 method)

## neighbor.jl -> cu_knn()

In [41]:
function cu_knn(Agg::Aggregate)
    # Definig Variables for calculing knn
    global Agg
    
    # Defining Coordinates of each cell on the aggregates
    Agg.Neighbor.i_Cell = reshape(
                repeat(
                    Agg.Position.X, 
                    size(Agg.Position.X ,1)
                ), 
                size(Agg.Position.X ,1), 
                size(Agg.Position.X ,1), 
                3
            ) - 
            reshape(
                repeat(
                    Agg.Position.X, 
                    inner=(size(Agg.Position.X ,1),1)
                ), 
                size(Agg.Position.X ,1), 
                size(Agg.Position.X ,1), 
                3
            )

    # Calculating Norm on every cell on the aggregate
    Agg.Neighbor.Dist = sqrt.(
                Agg.Neighbor.i_Cell[:,:,1] .^ 2 + 
                Agg.Neighbor.i_Cell[:,:,2] .^ 2 + 
                Agg.Neighbor.i_Cell[:,:,3] .^ 2
                )
    # # i_Cell = nothing; GC.gc(true)

    # Calculating index of knof each cell in the aggregate
    for i = 1:Agg.ParNeighbor.nn
        Agg.Neighbor.idx[i,:] = findmin(Agg.Neighbor.Dist; dims=1)[2]
        Agg.Neighbor.Dist[Agg.Neighbor.idx[i,:]] .= Inf
    end
    synchronize()
end

cu_knn (generic function with 2 methods)

In [42]:
CUDA.@time cu_knn(Agg)
Agg.Neighbor.idx

  0.012138 seconds (3.47 k CPU allocations: 276.094 KiB) (60 GPU allocations: 20.244 MiB, 10.39% memmgmt time)


12×538 CuArray{CartesianIndex{2}, 2, CUDA.Mem.DeviceBuffer}:
 CartesianIndex(1, 1)   CartesianIndex(2, 2)   …  CartesianIndex(538, 538)
 CartesianIndex(26, 1)  CartesianIndex(27, 2)     CartesianIndex(513, 538)
 CartesianIndex(3, 1)   CartesianIndex(4, 2)      CartesianIndex(534, 538)
 CartesianIndex(4, 1)   CartesianIndex(5, 2)      CartesianIndex(535, 538)
 CartesianIndex(30, 1)  CartesianIndex(31, 2)     CartesianIndex(507, 538)
 CartesianIndex(31, 1)  CartesianIndex(32, 2)  …  CartesianIndex(508, 538)
 CartesianIndex(2, 1)   CartesianIndex(1, 2)      CartesianIndex(537, 538)
 CartesianIndex(25, 1)  CartesianIndex(26, 2)     CartesianIndex(512, 538)
 CartesianIndex(27, 1)  CartesianIndex(28, 2)     CartesianIndex(514, 538)
 CartesianIndex(37, 1)  CartesianIndex(38, 2)     CartesianIndex(501, 538)
 CartesianIndex(77, 1)  CartesianIndex(78, 2)  …  CartesianIndex(461, 538)
 CartesianIndex(5, 1)   CartesianIndex(3, 2)      CartesianIndex(533, 538)

## run_event.jl -> simulate() -> rand_idx

In [43]:
Agg.Neighbor.rand_idx = getindex.(
                            Agg.Neighbor.idx, 1
                            )[getindex.(rand(
                                    2:Agg.ParNeighbor.nn,
                                    2*size(Agg.Position.X,1)
                                ),1),:]

1076×538 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 30  31  37  37  39   5  13  14  15  16  …  531  500  500  502  535  506  507
 77  78  44  30  33  87  37  51  52  53     449  493  506  509  456  460  461
 27  28  29   8  10  34  50   3   4   5     502  505  529  531  510  513  514
 27  28  29   8  10  34  50   3   4   5     502  505  529  531  510  513  514
  2   1   7   3   6  11  36   9  10  11     536  528  533  536  532  538  537
 27  28  29   8  10  34  50   3   4   5  …  502  505  529  531  510  513  514
 37  38  31   9  31  47  35   4   5   6     504  507  530  507  496  500  501
  4   5  36   2  38  40  12  13  14  15     527  499  538  501  503  534  535
 37  38  31   9  31  47  35   4   5   6     504  507  530  507  496  500  501
  2   1   7   3   6  11  36   9  10  11     536  528  533  536  532  538  537
 31  32   4  38   4  10   8   7   8   9  …  503  534  501  534  531  507  508
  5   3  84  32  46  28  91  36  37  38     488  453  508  495  514  535  533
 77  78  44  

## forces.jl -> cu_forces

In [62]:
function cui_force(t::Time, f::Contractile, Agg::Aggregate)
    # Definig Variables for calculing dX
    global Agg

    # Calculating distance for random forces (contractile)
    Agg.Force.r_p = Agg.Position.X .- 
                        Agg.Position.X[
                            Agg.Neighbor.rand_idx[
                                Int.(mod(
                                    t.t_f, size(Agg.Position.X, 1)
                                ) .+ 1),
                            :],
                        :]
    
    # Finding Distances/Norm for random forces
    Agg.Force.dist_p = sum(Agg.Force.r_p .^ 2, dims=2).^ 0.5

    # Finding distances
    Agg.Force.r = reshape(
            repeat(Agg.Position.X, inner=(Agg.ParNeighbor.nn,1)), 
            Agg.ParNeighbor.nn, size(Agg.Position.X)[1], 3
        ) .- 
        Agg.Position.X[getindex.(Agg.Neighbor.idx,1),:]

    # Finding Distances(Norm)
    Agg.Force.dist = ((sum(Agg.Force.r .^ 2, dims=3)) .^ 0.5)[:,:,1]

    # return(force(Agg.Force.dist))
    # # Finding forces for each cell
    # Agg.Force.F = force(Agg.Force.dist) .* Agg.Force.r ./ Agg.Force.dist

    # # Calculating de dX   -> dX[i,:] +=  r/dist * F
    # dX = sum(F[2:end,:,:]; dims=1)[1,:,:] - f.fₚ .* (r_p ./ dist_p)
    # synchronize()
end

cui_force (generic function with 2 methods)

In [71]:
force(qqq)

2×2 Matrix{Float64}:
 1.75872   1.26977
 0.777675  1.27689

In [67]:
cui_force(
    Time(0, 100000, 0.1),
    Contractile(0.00002),
    Agg
)
d = Agg.Force.dist
# force(d)

12×538 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 0.0      0.0      0.0      0.0      …  0.0      0.0      0.0      0.0
 1.99484  1.99484  1.99484  1.99484     1.99484  1.99484  1.99484  1.99484
 1.99822  1.99822  1.99822  1.99822     1.99822  1.99832  1.99822  1.99822
 1.99822  1.99822  1.99832  1.99822     1.99832  1.99832  1.99822  1.99822
 1.99832  1.99832  1.99832  1.99832     1.99832  2.0      1.99832  1.99832
 1.99832  1.99832  2.0      1.99832  …  2.0      2.00689  1.99832  1.99832
 2.0      2.0      2.00689  2.0         2.0      2.00689  2.0      2.0
 2.82478  2.82478  2.00689  2.0         2.00689  2.82478  2.82478  2.82478
 2.82478  2.82478  2.82478  2.00689     2.00689  2.82478  2.82478  2.82478
 2.82719  2.82719  2.82478  2.00689     2.82478  2.82719  2.82719  2.82719
 3.27     3.27     2.82719  2.82478  …  2.82478  3.27     3.27     3.27
 3.46308  3.46308  3.27     2.82478     2.82719  3.45707  3.46308  3.46308